In [1]:
from datetime import datetime

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import folium

In [2]:
df = pd.read_excel(io='../data.xlsx')

In [5]:
geodata = df[['num','date', 'owner', 'INN', 'N', 'E']]
geodata['month'] = geodata['date'].dt.to_period('M')

/tmp/ipykernel_33452/4061502996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geodata['month'] = geodata['date'].dt.to_period('M')


In [3]:
def coords(period):
    if period:
        start_date = period[0]
        end_date = period[1]
        mask = (geodata['month'] >= start_date) & (geodata['month'] <= end_date)
        return geodata[mask]
    else: return None

In [5]:
delta = (pd.to_datetime('01/01/2020').to_period('M'), pd.to_datetime('01/01/2022').to_period('M'))
coords(delta)

,num,date,owner,INN,N,E,month
118,АСТ00343НП,2020-02-14,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВЯЗО...",3.024000e+09,"48°06'26.152""N","45°35'44.941""Е",2020-02
162,АСТ16682НЭ,2020-10-01,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РОСС...",6.317130e+09,"47°06'0""N","47°42'0""Е",2020-10
167,БЛГ04004НП,2020-08-28,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДАЛЬ...",2.540255e+09,"54°17'4.0127""N","130°42'59.525""Е",2020-08
179,ВЛВ02660НП,2020-10-21,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДАЛЬ...",2.540255e+09,"46°39'26.2""N","134°26'14.5""Е",2020-10
688,ВЛГ02336НП,2020-03-18,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РОСС...",6.317130e+09,"50°46'0""N","44°21'0""Е",2020-03
...,...,...,...,...,...,...,...
15973,ЯКУ06987НП,2021-10-29,"Общество с ограниченной ответственностью ""Дюгю...",1.435352e+09,"72°17'11.601""N","117°38'0.553""Е",2021-10
15974,ЯКУ06988НП,2021-10-29,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДЮГЮ...",1.435352e+09,"72°01'11.604""N","116°29'25.208""Е",2021-10
15975,ЯКУ06989НП,2021-10-29,"Общество с ограниченной ответственностью ""Дюгю...",1.435352e+09,"72°12'52.615""N","116°35'38.269""Е",2021-10
16089,ЯКУ16667НП,2020-07-20,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,7.720330e+09,"63°26'28""N","129°33'49""Е",2020-07


In [ ]:
@interact(coords, dates=widgets.SelectionRangeSlider(
    index=
    options=
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}))

In [ ]:
len(geodata['month'][geodata['month'] > pd.to_datetime('2010-01-02').to_period('M')])

In [14]:
def on_value_change(change):
    print(change)

selection_range_slider = widgets.SelectionRangeSlider(
    options=geodata['month'][geodata['month'] > pd.to_datetime('2020-01-02').to_period('M')],
    index = (200,1000),
    description='Dates',
    orientation='horizontal',
    layout={'width': '1000px'})

selection_range_slider

SelectionRangeSlider(description='Dates', index=(108, 80), layout=Layout(width='1000px'), options=(Period('202…

In [ ]:
widgets.interact(on_value_change, change=selection_range_slider)

In [15]:
widgets.interact(coords, period=selection_range_slider)

interactive(children=(SelectionRangeSlider(description='Dates', index=(108, 632), layout=Layout(width='1000px'…

<function __main__.coords(period)>

Сделай селектИнт и поквартально